<a href="https://colab.research.google.com/github/yashsinghal11/Data-Science_-Theory-Coding_tutorial-/blob/main/Data_cleaning__practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('retail_store_sales.csv')

# -------------------------------
# 1. STANDARDIZE COLUMN NAMES
# -------------------------------
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(" ", "_")
)

# -------------------------------
# 2. STRIP SPACES IN STRING COLUMNS
# -------------------------------
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    df[col] = df[col].astype(str).str.strip()

# -------------------------------
# 3. HANDLE NUMERIC COLUMNS
# -------------------------------
num_cols = ["price_per_unit", "quantity", "total_spent"]

for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# -------------------------------
# 4. CLEAN "DISCOUNT_APPLIED" COLUMN
#     - Convert to True/False
#     - Fill NaN with False (common business logic)
# -------------------------------
df["discount_applied"] = (
    df["discount_applied"]
    .astype(str)
    .str.lower()
    .replace({"true": True, "false": False, "nan": np.nan})
)

df["discount_applied"] = df["discount_applied"].fillna(False)

# -------------------------------
# 5. HANDLE MISSING VALUES
# -------------------------------
df["item"] = df["item"].fillna("Unknown Item")

# Numeric fill with median
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# -------------------------------
# 6. FIX DATE FORMAT
# -------------------------------
df["transaction_date"] = pd.to_datetime(df["transaction_date"], errors="coerce")

# -------------------------------
# 7. ADD DERIVED COLUMNS
# -------------------------------
# Recompute total_spent if inconsistent
df["computed_total"] = df["price_per_unit"] * df["quantity"]

# if original missing or wrong, replace
df["total_spent"] = df.apply(
    lambda x: x["computed_total"] if pd.isna(x["total_spent"]) or abs(x["total_spent"] - x["computed_total"]) > 1 else x["total_spent"],
    axis=1
)

df.drop(columns=["computed_total"], inplace=True)

# -------------------------------
# 8. REMOVE DUPLICATES
# -------------------------------
df.drop_duplicates(inplace=True)

# -------------------------------
# 9. FINAL EXPORT
# -------------------------------
df.to_csv('retail_store_sales_cleanedfile.csv', index=False)

df.head()


/tmp/ipython-input-1077714446.py:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["discount_applied"] = df["discount_applied"].fillna(False)


,transaction_id,customer_id,category,item,price_per_unit,quantity,total_spent,payment_method,location,transaction_date,discount_applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,False
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False
